In [1]:
!pip install camelot-py[all]
!pip install tabula-py
!pip install PyPDF2
!apt-get install ghostscript

     |████████████████████████████████| 40 kB 23 kB/s 
     |████████████████████████████████| 77 kB 3.3 MB/s 
     |████████████████████████████████| 5.6 MB 39.1 MB/s 
     |████████████████████████████████| 11.7 MB 12.6 MB/s 
     |████████████████████████████████| 3.5 MB 27.4 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=f71b644e728ae011dda3722cce3630c1e9f31c33eba8d5f9636e31fcafda47d8
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
     |████████████████████████████████| 12.0 MB 4.0 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  

In [2]:
from google.colab import drive

drive.mount('/content/drive')
path  = '/content/drive/MyDrive/'

Mounted at /content/drive


In [3]:
import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re

#Para encontrar la página en la que se encuentra el dato.
def fnPDF_FindText(xFile, xString):
    # xfile : the PDF file in which to look
    # xString : the string to look for
    # import pyPdf, re
    PageFound = -1
    pdfDoc = pyPdf.PdfFileReader(xFile, "rb")
    for i in range(1, pdfDoc.getNumPages()):
        content = ""
        content += pdfDoc.getPage(i-1).extractText() + "\n"
        content1 = content#.lower()#content.encode('ascii', 'ignore').lower()
        ResSearch = re.search(xString, content1)
        if ResSearch is not None:
           PageFound = i 
           break
    return PageFound

In [4]:
import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re

#Para crear una clave de trimestre válida
def create_trimester_key(file_name):
    seed = "20"   
    element = file_name.split("_")[2]
    trimester = element[1]+element[2]
    year = element[2:4]
    return seed + year + trimester

In [5]:
import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re

#Para hacer la extraccion (SOLO PARA BALANCE)
def balance_general_activo_extraccion(path,filename,page,short_description,keyword):
    pagina = fnPDF_FindText(path+filename,"Estado de resultados")
    print("      Archvo encontrado en la página: ", pagina)

    print("      Paso 1: Limpieza general.")  
    tabla_leer = camelot.read_pdf(path+archivo,flavor="stream",pages=str(page))

    #Se eliminan las partes HORIZONATALES malas, para ello se hace la transpuesta y se trata sólo con "columnas".
    estado_de_resultados = tabla_leer[0].df.transpose()
    
    #El renglón 1 pasa a ser las "nuevas columnas", también se elimina un renglón con datos vacíos. 
    new_header = estado_de_resultados.iloc[0] #grab the first row for the header
    estado_de_resultados_2 = estado_de_resultados[1:] #take the data less the header row
    estado_de_resultados_2.columns = new_header #set the header row as the df header

    #Eliminando las columnas que están vacías, aplica para las que no tienen nombre como para las que no tienen contenido
    estado_de_resultados_2.drop([""], axis=1, inplace=True)

    #Así se identifican las que no tienen contenido
    estado_de_resultados_2.replace('', np.nan, inplace = True)
    estado_de_resultados_2.dropna(how='all', axis=1,inplace = True)

    #Se obtiene la transpuesta para continuar con la limpieza.
    estado_de_resultados_2 = estado_de_resultados_2.T
    estado_de_resultados_2.reset_index(inplace = True)
    
    print("      Paso 3: Limpieza minuciosa.")  
    #Se hace un reemplazo de los paréntesis y coma (,) para que se pueda hacer la transformación de los datos a numéricos.
    columnas = estado_de_resultados_2.columns
    for columna in columnas:
         estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace("(","") 
         estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace(")","")
         estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace(",","")

    #Se agregan algunos campos extra.
    estado_de_resultados_2["bank_name"] = "Santander"
    estado_de_resultados_2["table_name"] = "Balance General Activo"
    estado_de_resultados_2["trimester_date"] = create_trimester_key(filename)

    print("      Paso 4: Efectuando la suma.")  
    #El estado de resultados de Santander NO viene por trimestre pero está por "mes" que significa lo mismo
    estado_de_resultados_2['T_0_feature_value'] = estado_de_resultados_2[1] 
    estado_de_resultados_2['T_1_feature_value']	= estado_de_resultados_2[2]
    estado_de_resultados_2['T_2_feature_value'] = estado_de_resultados_2[3] 
    estado_de_resultados_2['T_3_feature_value'] = estado_de_resultados_2[4]
    estado_de_resultados_2['T_4_feature_value'] = estado_de_resultados_2[5]  

    lista_columnas_borrar = []

    #Dependiendo del archivo se eliminan las columnas innecesarias.
    if filename == "Reporte_Resultados_1T21.pdf":  
        #Se ponen las columnas a borrar
        lista_columnas_borrar = [1,2,3,4,5]

    elif filename == "Reporte_Resultados_3T20.pdf":
         lista_columnas_borrar = [1,2,3,4,5,6,7]

    elif filename == "Reporte_Resultados_4T20.pdf":
         lista_columnas_borrar = [1,2,3,4,5,6,7,8]

    else:
        
        #Se ponen las columnas a borrar
        lista_columnas_borrar = [1,2,3,4,5,6]

    #Después de la suma se eliminan todas las columnas que ya no sirven
    estado_de_resultados_2.drop(estado_de_resultados_2.columns[lista_columnas_borrar],axis=1,inplace=True)

    print("      Paso 5: Corrigiendo la descripción.")  
    #Se añade la descripción y se elimina una columna que tenía los nombres mal escritos de la descripción.
    estado_de_resultados_2['short_description'] = short_description
    estado_de_resultados_2.drop(estado_de_resultados_2.columns[0],axis=1,inplace=True)

    #Nota: hasta este punto TODAS LAS VARIABLES quedan como string para que se puedan despues convertir a entero.
    return estado_de_resultados_2

In [6]:
import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re

#Para hacer la extraccion (SOLO PARA BALANCE)
def balance_general_pasivo_extraccion(path,filename,page,short_description,keyword):
    pagina = fnPDF_FindText(path+filename,"Estado de resultados")
    print("      Archvo encontrado en la página: ", pagina)

    print("      Paso 1: Limpieza general.")  
    tabla_leer = ""

    #Caso especial para este archivo.
    if filename == "Reporte_Resultados_2T20.pdf":
        tabla_leer = camelot.read_pdf(path+archivo,flavor="stream",pages=str(page), edge_tol=500)

    else:
        tabla_leer = camelot.read_pdf(path+archivo,flavor="stream",pages=str(page))

    tabla_leer2=tabla_leer[0].df
    tabla_leer3 = tabla_leer2.copy()
    for i in range(0,len(tabla_leer2)):
      tabla_leer3[1][i] = tabla_leer2[1][i].replace("\n  \n","\n")
      tabla_leer3[1][i] = tabla_leer3[1][i].replace("\n \n","\n")

    #Se eliminan las partes HORIZONATALES malas, para ello se hace la transpuesta y se trata sólo con "columnas".
    estado_de_resultados = tabla_leer3.transpose()
    
    #El renglón 1 pasa a ser las "nuevas columnas", también se elimina un renglón con datos vacíos. 
    new_header = estado_de_resultados.iloc[0] #grab the first row for the header
    estado_de_resultados_2 = estado_de_resultados[1:] #take the data less the header row
    estado_de_resultados_2.columns = new_header #set the header row as the df header

    #Eliminando las columnas que están vacías, aplica para las que no tienen nombre como para las que no tienen contenido
    estado_de_resultados_2.drop([""], axis=1, inplace=True)

    #Así se identifican las que no tienen contenido
    estado_de_resultados_2.replace('', np.nan, inplace = True)
    estado_de_resultados_2.dropna(how='all', axis=1,inplace = True)

    #Se obtiene la transpuesta para continuar con la limpieza.
    estado_de_resultados_2 = estado_de_resultados_2.T
    estado_de_resultados_2.reset_index(inplace = True)
    
    #Caso especial para pasivo de este reporte
    if filename in ("Reporte_Resultados_2T20.pdf","Reporte_Resultados_3T20.pdf","Reporte_Resultados_4T20.pdf"):
        estado_de_resultados_2.drop(11, inplace=True)
        estado_de_resultados_2.drop(20, inplace=True)

    print("      Paso 3: Limpieza minuciosa.")  
    #Se hace un reemplazo de los paréntesis y coma (,) para que se pueda hacer la transformación de los datos a numéricos.
    columnas = estado_de_resultados_2.columns
    for columna in columnas:
         estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace("(","") 
         estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace(")","")
         estado_de_resultados_2[columna] = estado_de_resultados_2[columna].str.replace(",","")

    #Se agregan algunos campos extra.
    estado_de_resultados_2["bank_name"] = "Santander"
    estado_de_resultados_2["table_name"] = "Balance General Pasivo"
    estado_de_resultados_2["trimester_date"] = create_trimester_key(filename)

    print("      Paso 4: Efectuando la suma.")  
    #El estado de resultados de Santander NO viene por trimestre pero está por "mes" que significa lo mismo
    estado_de_resultados_2['T_0_feature_value'] = estado_de_resultados_2[1] 
    estado_de_resultados_2['T_1_feature_value']	= estado_de_resultados_2[2]
    estado_de_resultados_2['T_2_feature_value'] = estado_de_resultados_2[3]
    estado_de_resultados_2['T_3_feature_value'] = estado_de_resultados_2[4] 
    estado_de_resultados_2['T_4_feature_value'] = estado_de_resultados_2[5] 

    lista_columnas_borrar = []

    #Dependiendo del archivo se eliminan las columnas innecesarias.
    if filename == "Reporte_Resultados_1T21.pdf":  
        #Se ponen las columnas a borrar
        lista_columnas_borrar = [1,2,3,4,5]

    elif filename == "Reporte_Resultados_3T20.pdf":
         lista_columnas_borrar = [1,2,3,4,5,6,7]

    elif filename == "Reporte_Resultados_4T20.pdf":
         lista_columnas_borrar = [1,2,3,4,5,6,7,8]

    else:
        
        #Se ponen las columnas a borrar
        lista_columnas_borrar = [1,2,3,4,5,6]

    #Después de la suma se eliminan todas las columnas que ya no sirven
    estado_de_resultados_2.drop(estado_de_resultados_2.columns[lista_columnas_borrar],axis=1,inplace=True)

    print("      Paso 5: Corrigiendo la descripción.")  
    #Se añade la descripción y se elimina una columna que tenía los nombres mal escritos de la descripción.
    estado_de_resultados_2['short_description'] = short_description
    estado_de_resultados_2.drop(estado_de_resultados_2.columns[0],axis=1,inplace=True)

    #Nota: hasta este punto TODAS LAS VARIABLES quedan como string para que se puedan despues convertir a entero.
    return estado_de_resultados_2

In [8]:
#Comienza el proceso PARA EL ACTIVO

import camelot 
import tkinter
import ghostscript
import numpy as np
import pandas as pd
import PyPDF2 as pyPdf, re


#De todas las columnas que mostró Samuel, éstas son las que Santander tiene EN EL BALANCE GENERAL ACTIVOS.
"""
1 DISPONIBILIDADES
2 Cuentas de margen
3 INVERSIONES EN VALORES
4 Títulos para negociar 
5 Títulos disponibles para la venta 
6 Títulos conservados a vencimiento 
7 Deudores por reporto
8 Derivados 
9 Con Fines de Negociación
10 Con fines de cobertura
11 Ajustes de valuación por cobertura de activos financieros
Total Cartera de crédito vigente
12 Créditos comerciales
13 Actividad empresarial o comercial
14 Entidades financieras
15 Entidades gubernamentales
16 Créditos de consumo
17 Créditos a la vivienda
18 Media y residencial
19 De interés social
Total Cartera de crédtio vencida
20 Créditos Comerciales
Actividad empresarial o comercial
Créditos de consumo
Créditos a la vivienda
Media y residencial
De interés social
Total cartera de crédito (neto)
Estimación preventiva para riesgos. crediticios
Total de cartera de crédtio (neto)
Deudores por primas(neto)
Cuentas por cobrar a reaseguradores y reafianzadores (neto)
Beneficios por recibir en operaciones de bursatilización
Otras cuentas por cobrar(neto)
Bienes adjudicados(neto)
Inmuebles,Mobiliario y equipo(neto)
Inversiones permanentes
Impuestos y PTU diferidos(neto)
Otros activos
Cargos diferidos, pagos anticipados e intangibles
Total Activo
"""

#Se agregan/modifican éstas porque no estaban en el listado general. 
"""
Agregados
Entidades financieras 2

Cambiados 
otros activos y blablabla por solo otros activos
diferido por Impuestos y PTU diferidos(neto)
"""


#********************************************************************************************
#De todas las columnas que mostró Samuel, éstas son las que Santander tiene EN EL BALANCE GENERAL PASIVOS.

"""
Fecha de actualización de datos
Fecha de carga
Fecha de trimestre
Nombre de banco
CAPTACIÓN TRADICIONAL
Depóstios de exigibilidad inmediata
Depóstios a plazo del publico general
Depóstios a plazo del mercado de dinero
Títulos de crédito emitidos
Cuenta Global de captación sin movimientos
PRESTAMOS INTERBANCARIOS Y DE OTROS ORGANISMOS
 De exigiilibad inmediata
 De corto plazo
 De largo plazo
Reservas técnicas 
Acreedores por reporto
Prestamos de valores
COLATERALES VENDIDOS O DADOS EN GARANTIA
 Reportos
 Préstamo de valores 
DERIVADOS
Con fines de Negociación
Con fines de Cobertura
Ajustes de valuación por cobertura de pasivos financieros(neto)
Cuentas por pagar a reaseguradores y reafianzadores (neto)
OTRAS CUENTAS POR PAGAR
 Participación de los trabajadores en las utilidades por pagar
 Acreedores por liquidación de operaciones
 Acreedores por colaterales recibidos en efectivo
 Acreedores diversos y otras cuentas por pagar
Obligaciones subordinadas en circulación
Créditos diferidos y cobros anticipados
TOTAL PASIVO
CAPITAL CONTRIBUIDO
Capital social
Prima en venta de acciones
CAPITAL GANADO
Reservas de capital 
Resultado de ejercicios anteriores
Resultado por valuación de títulos disponibles para la venta
Resultado por valuación de instrumentos de cobertura de flujos de efect
Efecto acumulado por conversión
Remediciones por beneficios definidos a los empreados
Resultado neto
CAPITAL CONTABLE MAYORITARIO
Participación no controlada
TOTAL CAPITAL CONTABLE
TOTAL PASIVO Y CAPITAL CONTABLE
"""

"""
cambiados
Captación por captación tradicional
"""


#Lista de las descripciones finales para activo.
short_description_activo = [
                             'Disponibilidades',
                             'Cuentas de margen',
                             'Inversiones en valores',
                             'Titulos para negociar',
                             'Titulos disponibles para la venta',
                             'Titulos conservados a vencimiento',
                             'Deudores por reporto',
                             'Derivados',
                             'Con fines de negociación',
                             'Con fines de cobertura',
                             'Ajustes de valuacion por cobertura de activos financieros',
                             'Creditos comerciales',
                             'Actividad empresarial o comercial',
                             'Entidades financieras',
                             'Entidades gubernamentales',
                             'Creditos de consumo',
                             'Creditos a la vivienda',
                             'Media y residencial',
                             'De interes social',
                             'Creditos adquiridos al Infonavit o el Fovissste',
                             'Total cartera de credito vigente',
                             'Creditos comerciales',
                             'Actividad empresarial o comercial',
                             'Entidades financieras 2',
                             'Creditos de consumo',
                             'Creditos a la vivienda',
                             'Media y residencial',
                             'De interés social',
                             'Créditos adquiridos al Infonavit o el Fovissste',
                             'Total cartera de crédito vencida',
                             'Total cartera de crédito',
                             'Estimacion preventiva para riesgos crediticios',
                             'Cartera de crédito neto',
                             'Beneficios por recibir en operaciones de bursatilización',
                             'Otras cuentas por cobrar neto',
                             'Bienes adjudicados neto',
                             'Propiedades mobiliario y equipo neto',
                             'Inversiones permanentes',
                             'Impuestos y PTU diferidos (neto)',
                             'Cargos diferidos pagos anticipados e intangibles',
                             'Otros activos',
                             'Total activo'
                          ]

#Lista de las descripciones finales para pasivo.
short_description_pasivo  = [
                              'Captacion tradicional',
                              'Depositos de exigibilidad inmediata',
                              'Depositos a plazo - publico en general',
                              'Depositos a plazo - mercado de dinero',
                              'Titulos de credito emitidos',
                              'Cuenta global de captacion sin movimientos',
                              'Prestamos bancarios y de otros organismos',
                              'De exigibilidad inmediata',
                              'De corto plazo',
                              'De largo plazo',
                              'Acreedores por reporto',
                              'Colaterales vendidos o dados en garantía',
                              'Reportos',
                              'Prestamo de valores',
                              'Derivados',
                              'Con fines de negociacion',
                              'Con fines de cobertura',
                              'Ajuste de valuacion por cobertura de pasivos financieros',
                              'Otras cuentas por pagar',
                              'Participacion de los trabajadores en las utilidades por pagar',
                              'Acreedores por liquidación de operaciones',
                              'Acreedores por cuentas de margen',
                              'Acreedores por colaterales recibidos en efectivo',
                              'Acreedores diversos y otras cuentas por pagar',
                              'Obligaciones subordinadas en circulacion',
                              'Creditos diferidos y cobros anticipados',
                              'Total pasivo',
                              'Capital contribuido',
                              'Capital social',
                              'Prima en venta de acciones',
                              'Capital ganado',
                              'Reservas de capital',
                              'Resultado de ejercicios anteriores',
                              'Resultado por valuacion de titulos disponibles para la venta',
                              'Resultado por valuación de instrumentos de cobertura de flujos de efectivo', #agregado manualmente
                              'Efecto acumulado por conversión',
                              'Remediciones por beneficios definidos a los empleados',
                              'Resultado neto',
                              'Participacion no controladora',
                              'Total capital contable',
                              'Total pasivo y capital contable'
                            ]

#Nombres de los archivos.
#Aquí el método de la búsqueda de página no 
#funciona por lo que se genera una lista de 
#listas con los siguientes elementos:
# posición 0 -> nombre archivo
# posición 1 -> página balance general activo
# posición 2 -> página balance general pasivo 
lista_archivos = [
                  #["Reporte_Resultados_1T21.pdf",40,41],
                  #["Reporte_Resultados_2T20.pdf",41,42],
                  ["Reporte_Resultados_2T21.pdf",42,43],
                  #["Reporte_Resultados_3T20.pdf",41,42],
                  #["Reporte_Resultados_4T20.pdf",38,39],                 
                 ]

path  = "/content/drive/MyDrive/hackatonBBVA2021/estadosFinancieros/Santander/"

list_of_dataframes_activo = []
list_of_dataframes_pasivo = []

In [9]:
outputpath = "/content/drive/MyDrive/hackatonBBVA2021/csv/Santander/"

In [10]:
#Para activos
for elemento in lista_archivos:
    archivo = elemento[0]
    pagina = elemento[1]
    print("******* Ahora ejecutando archivo: ", archivo)
    df_provisional = balance_general_activo_extraccion(path,archivo,pagina,short_description_activo,"Estado de resultados")
    list_of_dataframes_activo.append(df_provisional)

final_dataframe_activo = pd.concat(list_of_dataframes_activo)
print(final_dataframe_activo)
final_dataframe_activo.to_csv(outputpath+"balance_general_activo_santander.csv",index=False)

******* Ahora ejecutando archivo:  Reporte_Resultados_2T21.pdf
      Archvo encontrado en la página:  7
      Paso 1: Limpieza general.


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:4174]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:4389]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:29]


      Paso 3: Limpieza minuciosa.
      Paso 4: Efectuando la suma.
      Paso 5: Corrigiendo la descripción.
    bank_name  ...                                  short_description
0   Santander  ...                                   Disponibilidades
1   Santander  ...                                  Cuentas de margen
2   Santander  ...                             Inversiones en valores
3   Santander  ...                              Titulos para negociar
4   Santander  ...                  Titulos disponibles para la venta
5   Santander  ...                  Titulos conservados a vencimiento
6   Santander  ...                               Deudores por reporto
7   Santander  ...                                          Derivados
8   Santander  ...                           Con fines de negociación
9   Santander  ...                             Con fines de cobertura
10  Santander  ...  Ajustes de valuacion por cobertura de activos ...
11  Santander  ...                               C

In [11]:
#*****************************************************************************************
#Para pasivos
for elemento in lista_archivos:
    archivo = elemento[0]
    pagina = elemento[2]
    print("******* Ahora ejecutando archivo: ", archivo)
    df_provisional = balance_general_pasivo_extraccion(path,archivo,pagina,short_description_pasivo,"Estado de resultados")
    list_of_dataframes_pasivo.append(df_provisional)

final_dataframe_pasivo = pd.concat(list_of_dataframes_pasivo)
print(final_dataframe_pasivo)
final_dataframe_pasivo.to_csv(outputpath+"balance_general_pasivo_santander.csv",index=False)

******* Ahora ejecutando archivo:  Reporte_Resultados_2T21.pdf
      Archvo encontrado en la página:  7
      Paso 1: Limpieza general.


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:4174]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:4389]
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:42]


      Paso 3: Limpieza minuciosa.
      Paso 4: Efectuando la suma.
      Paso 5: Corrigiendo la descripción.
    bank_name  ...                                  short_description
0   Santander  ...                              Captacion tradicional
1   Santander  ...                Depositos de exigibilidad inmediata
2   Santander  ...             Depositos a plazo - publico en general
3   Santander  ...              Depositos a plazo - mercado de dinero
4   Santander  ...                        Titulos de credito emitidos
5   Santander  ...         Cuenta global de captacion sin movimientos
6   Santander  ...          Prestamos bancarios y de otros organismos
7   Santander  ...                          De exigibilidad inmediata
8   Santander  ...                                     De corto plazo
9   Santander  ...                                     De largo plazo
10  Santander  ...                             Acreedores por reporto
11  Santander  ...           Colaterales vendidos 

In [13]:
final_dataframe_activo

,bank_name,table_name,trimester_date,T_0_feature_value,T_1_feature_value,T_2_feature_value,T_3_feature_value,T_4_feature_value,short_description
0,Santander,Balance General Activo,2021T2,113843,97483,94802,83502,85658,Disponibilidades
1,Santander,Balance General Activo,2021T2,5209,4017,4122,3876,4638,Cuentas de margen
2,Santander,Balance General Activo,2021T2,458685,558857,575415,552369,540938,Inversiones en valores
3,Santander,Balance General Activo,2021T2,122957,204194,208089,173679,175739,Titulos para negociar
4,Santander,Balance General Activo,2021T2,324353,343332,356039,367422,353978,Titulos disponibles para la venta
5,Santander,Balance General Activo,2021T2,11375,11331,11287,11268,11221,Titulos conservados a vencimiento
6,Santander,Balance General Activo,2021T2,39505,47713,62294,21813,36848,Deudores por reporto
7,Santander,Balance General Activo,2021T2,200890,228247,304687,324984,385504,Derivados
8,Santander,Balance General Activo,2021T2,193024,218097,296381,315071,376412,Con fines de negociación
9,Santander,Balance General Activo,2021T2,7866,10150,8306,9913,9092,Con fines de cobertura


In [ ]:
final_dataframe_pasivo

,bank_name,table_name,trimester_date,T_0_feature_value,T_1_feature_value,T_2_feature_value,T_3_feature_value,T_4_feature_value,short_description
0,Santander,Balance General Pasivo,20211T,848345,845107,862618,880490,861518,Captacion tradicional
1,Santander,Balance General Pasivo,20211T,536791,515449,507949,506332,529554,Depositos de exigibilidad inmediata
2,Santander,Balance General Pasivo,20211T,183904,195736,199815,223703,216206,Depositos a plazo - publico en general
3,Santander,Balance General Pasivo,20211T,45094,51467,63483,58072,63040,Depositos a plazo - mercado de dinero
4,Santander,Balance General Pasivo,20211T,80718,80663,89634,90750,51178,Titulos de credito emitidos
...,...,...,...,...,...,...,...,...,...
38,Santander,Balance General Pasivo,20204T,1523,1009,1001,995,986,Remediciones por beneficios definidos a los em...
39,Santander,Balance General Pasivo,20204T,20154,14674,9644,5414,21332,Resultado neto
40,Santander,Balance General Pasivo,20204T,39,32,36,43,42,Participacion no controladora
41,Santander,Balance General Pasivo,20204T,158871,151475,146536,141041,138695,Total capital contable
